# 🚀 自研动态图学习框架 AdaptoFlux：在 Titanic 上实现全自动建模

欢迎使用 **AdaptoFlux** —— 一种基于函数图自动生长的可解释机器学习框架。

本 Notebook 演示如何：
- 加载预处理数据
- 训练动态函数图模型
- 保存图结构为 JSON
- 可视化图结构（交互式）
- 生成 Kaggle 提交文件

👉 所有代码开源，欢迎 Star & Fork！

In [ ]:
! git init
! git clone https://github.com/gugugu12138/AdaptoFlux

In [ ]:
! pip install -r /kaggle/working/AdaptoFlux/requirements.txt


In [ ]:
cd /kaggle/working/AdaptoFlux

In [ ]:
import pandas as pd
import numpy as np
import json
import traceback
from ATF.core.adaptoflux import AdaptoFlux
from ATF.CollapseManager.collapse_functions import CollapseMethod
from ATF.ModelTrainer.LayerGrowTrainer.layer_grow_trainer import LayerGrowTrainer

import logging

logging.basicConfig(
    level=logging.INFO,
    format='[%(levelname)s] %(name)s: %(message)s'
)

logging.info("✅ Logging 测试：配置成功！")

## 🧩 自定义方法池（methods.py）

确保 `methods.py` 已定义基础函数（如 `add_values`, `multiply_values` 等），并用 `@method_profile` 装饰。示例的方法池在/kaggle/working/AdaptoFlux/examples/kaggle/titanic/methods.py中

内容为：

In [ ]:
# 该代码块无需运行
import math
from ATF.methods.decorators import method_profile


@method_profile(
    output_count=1,
    input_types=['scalar'],
    output_types=['scalar'],
    group="basic",           # ✅ 统一分组
    weight=1.0,
    vectorized=False
)
def return_value(x):
    """
    返回原数值
    :param x: 输入数值
    :return: 原数值
    """
    return [x]

@method_profile(
    output_count=1,
    input_types=['scalar', 'scalar'],
    output_types=['scalar'],
    group="basic",           # ✅ 统一分组
    weight=1.0,
    vectorized=False
)
def add_values(x, y):
    """
    将两个数值相加
    :param x: 第一个数
    :param y: 第二个数
    :return: 两数之和
    """
    return [x + y]

@method_profile(
    output_count=1,
    input_types=['scalar', 'scalar'],
    output_types=['scalar'],
    group="basic",           # ✅ 统一分组
    weight=1.0,
    vectorized=False
)
def calculate_difference(a, b):
    """
    计算两个数的差
    :param a: 被减数
    :param b: 减数
    :return: 差值
    """
    return [a - b]


@method_profile(
    output_count=1,
    input_types=['scalar', 'scalar'],
    output_types=['scalar'],
    group="basic",           # ✅ 统一分组
    weight=1.0,
    vectorized=False
)
def multiply_values(x, y):
    """
    将两个数值相乘
    :param x: 第一个数
    :param y: 第二个数
    :return: 两数之积
    """
    return [x * y]

@method_profile(
    output_count=1,
    input_types=['scalar', 'scalar'],
    output_types=['scalar'],
    group="basic",
    weight=1.0,
    vectorized=False
)
def divide_values(x, y):
    """
    将两个数值相除（带零检查）
    :param x: 被除数
    :param y: 除数
    :return: x / y 或 0（当y=0时）
    """
    return [x / y if y != 0 else 0]

@method_profile(
    output_count=2,
    input_types=['scalar'],
    output_types=['scalar', 'scalar'],
    group="basic",           # ✅ 统一分组
    weight=1.0,
    vectorized=False
)
def return_two_values(x):
    """
    返回两个原数值
    :param x: 输入数值
    :return: 原数值
    """
    return [x, x]

# 没提升没降低，只会拉低速度和计算量
# @method_profile(
#     output_count=3,
#     input_types=['scalar'],
#     output_types=['scalar', 'scalar', 'scalar'],
#     group="basic",           # ✅ 统一分组
#     weight=1.0,
#     vectorized=False
# )
# def return_three_values(x):
#     """
#     返回两个原数值
#     :param x: 输入数值
#     :return: 原数值
#     """
#     return [x, x, x]

@method_profile(
    output_count=1,
    input_types=['scalar'],
    output_types=['scalar'],
    group="basic",           # 统一归入 basic 组
    weight=1.0,
    vectorized=False
)
def decrement(x):
    """
    将输入值减 1
    :param x: 输入数值
    :return: x - 1
    """
    return [x - 1]


@method_profile(
    output_count=1,
    input_types=['scalar'],
    output_types=['scalar'],
    group="basic",           # 统一归入 basic 组
    weight=1.0,
    vectorized=False
)
def increment(x):
    """
    将输入值加 1
    :param x: 输入数值
    :return: x + 1
    """
    return [x + 1]


@method_profile(
    output_count=1,
    input_types=['scalar'],
    output_types=['scalar'],
    group="basic",           # 统一归入 basic 组
    weight=1.0,
    vectorized=False
)
def negate_value(x):
    """
    将输入值取相反数
    :param x: 输入数值
    :return: -x
    """
    return [-x]

## 🗃️ 加载 Titanic 预处理数据

In [ ]:
def load_titanic_for_adaptoflux(train_processed_path, methods_path=None, collapse_method=CollapseMethod.SUM):
    df = pd.read_csv(train_processed_path)
    if 'Survived' not in df.columns:
        raise ValueError("train_processed.csv 必须包含 'Survived' 列")
    labels = df['Survived'].values
    values = df.drop(columns=['Survived']).values
    values = np.array(values, dtype=np.float64)
    return AdaptoFlux(
        values=values,
        labels=labels,
        methods_path=methods_path,
        collapse_method=collapse_method
    )

# 加载模型
model = load_titanic_for_adaptoflux(
    train_processed_path='examples/kaggle/titanic/output/train_processed.csv',
    methods_path='examples/kaggle/titanic/methods.py',
    collapse_method=CollapseMethod.SUM
)

print(f"✅ 加载数据完成：{model.values.shape[0]} 个样本，{model.values.shape[1]} 个特征")

## 🌱 添加自定义坍缩函数

In [ ]:
def collapse_sum_positive(values):
    total = np.sum(values)
    return 1 if total > 0 else 0

model.add_collapse_method(collapse_sum_positive)

## 🏗️ 模型训练（逐层生长）

In [ ]:
trainer = LayerGrowTrainer(
    adaptoflux_instance=model,
    max_attempts=10,
    decision_threshold=0.0,
    verbose=True
)

trainer.train(
    input_data=model.values,
    target=model.labels,
    max_layers=15,
    save_model=True,
    on_retry_exhausted="rollback",
    rollback_layers=2,
    max_total_attempts=2000
)

## 🎨 可视化图结构

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# 读取图
G = nx.read_gexf("/kaggle/working/AdaptoFlux/models/best/graph.gexf")

# 指定根节点（请根据你的数据修改 root）
# 如果不知道 root，可以选择一个度最高的节点，或手动指定
root = "root"  # 替换为你的实际 root 节点名
if root not in G:
    # 如果没有明确的 root，可以用一个中心节点（例如度最大的节点）
    root = max(dict(G.degree()), key=lambda x: dict(G.degree())[x])
    print(f"使用度最大的节点作为根: {root}")

# 使用 BFS 计算每个节点到 root 的距离（层数）
try:
    # 对于有向图，可能需要转为无向图来 BFS
    if G.is_directed():
        bfs_dist = nx.shortest_path_length(G.to_undirected(), source=root)
    else:
        bfs_dist = nx.shortest_path_length(G, source=root)
except nx.NetworkXNoPath:
    print("图不连通，只考虑包含 root 的连通分量")
    # 只保留 root 所在的连通分量
    if G.is_directed():
        connected_nodes = nx.node_connected_component(G.to_undirected(), root)
    else:
        connected_nodes = nx.node_connected_component(G, root)
    G = G.subgraph(connected_nodes)
    bfs_dist = nx.shortest_path_length(G.to_undirected() if G.is_directed() else G, source=root)

# 按距离分层
layers = {}
for node, dist in bfs_dist.items():
    layers.setdefault(dist, []).append(node)

# 手动设置布局：x 坐标在每层内均匀分布，y 坐标为层数（距离）
pos = {}
for layer, nodes in layers.items():
    pos.update({node: (i, -layer) for i, node in enumerate(nodes)})  # y = -layer 保证 root 在顶部

# 设置节点颜色（可选：根据层设置不同颜色）
node_colors = [bfs_dist[node] for node in G.nodes]

# 绘图
plt.figure(figsize=(12, 8))
nx.draw(
    G, pos,
    with_labels=True,
    node_color=node_colors,
    cmap='viridis',
    node_size=600,
    font_size=6,
    font_color='black',
    edge_color='gray',
    arrows=True if G.is_directed() else False,
    width=1.0,
    alpha=0.9
)

plt.title(f"Hierarchical Layout from Root: {root}")
plt.axis('off')
plt.tight_layout()
plt.show()

## 📊 模型评估

In [ ]:
def _evaluate_accuracy(output: np.ndarray, target: np.ndarray) -> float:
    try:
        if len(output.shape) == 1 or output.shape[1] == 1:
            pred_classes = (output >= 0.5).astype(int).flatten()
        else:
            pred_classes = np.argmax(output, axis=1)
        true_labels = np.array(target).flatten()
        return np.mean(pred_classes == true_labels)
    except Exception as e:
        print(f"评估失败: {e}")
        return 0.0

pred = model.infer_with_graph(model.values)
acc = _evaluate_accuracy(pred, model.labels)
print(f"🎯 最终准确率: {acc:.4f}")

## 📤 生成 Kaggle 提交文件

In [ ]:
import pandas as pd
import os

def load_titanic_for_adaptoflux(train_processed_path, methods_path=None, collapse_method=CollapseMethod.SUM):
    """
    从预处理后的 Titanic 训练集 CSV 加载数据，并转换为 AdaptoFlux 可用的格式。

    :param train_processed_path: 预处理后的 train_processed.csv 文件路径
    :param methods_path: 方法路径（传给 AdaptoFlux）
    :param collapse_method: 坍缩方法（传给 AdaptoFlux）
    :return: AdaptoFlux 实例
    """
    # 读取 CSV
    df = pd.read_csv(train_processed_path)

    # 确保存在 Survived 列
    if 'Survived' not in df.columns:
        raise ValueError("train_processed.csv 必须包含 'Survived' 列作为标签")

    # 分离标签和特征
    labels = df['Survived'].values  # 一维标签
    values = df.drop(columns=['Survived']).values  # 二维特征矩阵

    # 转换为 numpy 浮点类型（防止 int64/float64 混合类型问题）
    values = np.array(values, dtype=np.float64)

    # 创建 AdaptoFlux 实例
    adaptoflux_instance = AdaptoFlux(
        values=values,
        labels=labels,
        methods_path=methods_path,
        collapse_method=collapse_method
    )

    return adaptoflux_instance

model = load_titanic_for_adaptoflux(train_processed_path='examples/kaggle/titanic/output/test_processed.csv',
                                    methods_path='examples/kaggle/titanic/methods.py')

model.add_collapse_method(collapse_sum_positive)

model.load_model(folder='models/best')

pred = model.infer_with_graph(model.values)

# 生成对应的 PassengerId，从 892 开始
passenger_ids = range(892, 892 + len(pred))

# 构建 DataFrame
submission = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived': pred
})

submission.to_csv('/kaggle/working/submission.csv', index=False)

print("✅ 提交文件已生成：/kaggle/working/submission.csv")

print("📁 当前 /kaggle/working 下的文件：")
print(os.listdir("/kaggle/working"))

display(submission.head())

## 🙏 致谢

感谢 Kaggle 提供 Titanic 数据集。

如果你喜欢这个项目，请：
- ⭐ Star 仓库
- 💬 在评论区分享你的想法！